## CoreIR

This notebook uses the `"coreir"` mantle backend on the icestick.

We begin by building a normal `Magma` circuit using `Mantle` 
and the `Loam` `IceStick` board. 

In [1]:
import magma as m
# default mantle target is coreir, so no need to do this unless you want to be explicit
# m.set_mantle_target("coreir")

In [2]:
from mantle import Counter
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.D5.on()

N = 22
main = icestick.main()

counter = Counter(N)
m.wire(counter.O[N-1], main.D5)

m.EndCircuit()

import lattice ice40


To compile to `coreir`, we simply set the `output` parameter to the `m.compile` command to `"coreir"`.

In [3]:
m.compile("build/blink_coreir", main, output="coreir")

We can inspect the generated `.json` file.

In [4]:
%cat build/blink_coreir.json

{"top":"global.main",
"namespaces":{
  "global":{
    "modules":{
      "Add22_cout":{
        "type":["Record",[
          ["I0",["Array",22,"BitIn"]],
          ["I1",["Array",22,"BitIn"]],
          ["O",["Array",22,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "bit_const_0_None":{
            "modref":"corebit.const",
            "modargs":{"value":["Bool",false]}
          },
          "coreir_add23_inst0":{
            "genref":"coreir.add",
            "genargs":{"width":["Int",23]}
          }
        },
        "connections":[
          ["coreir_add23_inst0.in0.22","bit_const_0_None.out"],
          ["coreir_add23_inst0.in1.22","bit_const_0_None.out"],
          ["self.I0.0","coreir_add23_inst0.in0.0"],
          ["self.I0.10","coreir_add23_inst0.in0.10"],
          ["self.I0.11","coreir_add23_inst0.in0.11"],
          ["self.I0.12","coreir_add23_inst0.in0.12"],
          ["self.I0.13","coreir_add23_inst0.in0.13"],
          ["self.I0.14","corei

We can use the `coreir` command line tool to generate verilog.

In [5]:
%%bash
coreir -i build/blink_coreir.json -o build/blink_coreir.v

/Users/hanrahan/git/coreir/src/binary/coreir.cpp:188 Running Runningvpasses
/Users/hanrahan/git/coreir/src/passes/transform/rungenerators.cpp:10 In Run Generators
/Users/hanrahan/git/coreir/src/passes/transform/rungenerators.cpp:26 Done running generators
/Users/hanrahan/git/coreir/src/binary/coreir.cpp:197 Running vpasses
/Users/hanrahan/git/coreir/src/binary/coreir.cpp:238 Modified?: No


And now we can inspect the generated verilog from coreir, notice that includes the `verilog` implementations of all the coreir primitives.

In [6]:
%cat build/blink_coreir.v

module coreir_reg #(parameter width = 1, parameter clk_posedge = 1, parameter init = 1) (input clk, input [width-1:0] in, output [width-1:0] out);
  reg [width-1:0] outReg=init;
  wire real_clk;
  assign real_clk = clk_posedge ? clk : ~clk;
  always @(posedge real_clk) begin
    outReg <= in;
  end
  assign out = outReg;
endmodule

module coreir_const #(parameter width = 1, parameter value = 1) (output [width-1:0] out);
  assign out = value;
endmodule

module coreir_add #(parameter width = 1) (input [width-1:0] in0, input [width-1:0] in1, output [width-1:0] out);
  assign out = in0 + in1;
endmodule

module corebit_const #(parameter value = 1) (output out);
  assign out = value;
endmodule

module Add22_cout (output COUT, input [21:0] I0, input [21:0] I1, output [21:0] O);
wire bit_const_0_None_out;
wire [22:0] coreir_add23_inst0_out;
corebit_const #(.value(0)) bit_const_0_None(.out(bit_const_0_None_out));
coreir_add #(.width(23)) coreir_add23_inst0(.in0({bit_const_0_None_out,I0[21],I0[2

In [7]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif blink_coreir.blif' blink_coreir.v
arachne-pnr -q -d 1k -o blink_coreir.txt -p blink_coreir.pcf blink_coreir.blif 
icepack blink_coreir.txt blink_coreir.bin
#iceprog blink_coreir.bin

/Users/hanrahan/git/magmathon/notebooks/advanced/build
